In [1]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2

In [2]:
model = YOLO("yolov3.pt")

PRO TIP 💡 Replace 'model=yolov3.pt' with new 'model=yolov3u.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [3]:
# Use the model
model.train(data="VisDrone.yaml", epochs=50, batch=8)  # train the model


Ultralytics YOLOv8.0.98 🚀 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12000MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov3.pt, data=VisDrone.yaml, epochs=50, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, ops

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.4
                   all        548      38759      0.437      0.336      0.331      0.197

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       5/50      9.62G      1.352     0.9889     0.9584        558        640: 100%|██████████| 809/809 [07:09<00:00,  1.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:08<00:00,  4.2
                   all        548      38759      0.446      0.336      0.347      0.209

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       6/50      8.82G      1.327     0.9563     0.9516        590        640: 100%|██████████| 809/809 [07:44<00:00,  1.74it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.4
          

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.9
                   all        548      38759      0.539      0.417      0.434      0.266

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      24/50      8.55G      1.145     0.7308     0.9005        474        640: 100%|██████████| 809/809 [06:54<00:00,  1.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.9
                   all        548      38759      0.565      0.416      0.441      0.271

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      25/50      8.58G      1.138     0.7248     0.9005        609        640: 100%|██████████| 809/809 [06:54<00:00,  1.95it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:07<00:00,  4.9
          

In [5]:
from PIL import Image
import io
import pathlib

with open("IMG-20230510-WA0000.jpg", "rb") as f:
    image = Image.open(io.BytesIO(f.read()))
    image = image.resize((1024, 720))
    image = np.asarray(image)

In [6]:
results = model.predict(image)


In [7]:
print(results[0].boxes.data)

In [8]:
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
  lw = max(round(sum(image.shape) / 2 * 0.003), 2)
  p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
  cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
  if label:
    tf = max(lw - 1, 1)  # font thickness
    w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # text width, height
    outside = p1[1] - h >= 3
    p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
    cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(image, label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2), 0, 
                lw / 3, txt_color, thickness=tf, lineType=cv2.LINE_AA)

In [9]:
# cv2.imwrite(filename, img)

In [10]:
def plot_bboxes(image, boxes, labels=[], colors=[], score=True, conf=None):
    #Define COCO Labels
    if labels == []:
        labels = {0: u'__background__', 1: u'pedestrian', 2: u'people',3: u'bicycle', 4: u'car', 5: u'van', 
              6: u'truck', 7: u'tricycle', 8: u'awning-tricycle', 9: u'bus', 10: u'motor', 11: u''}
    #Define colors
    if colors == []:
        colors = [(89, 161, 197), (67, 161, 255), (19, 222, 24), (186, 55, 2), (167, 146, 11), (190, 76, 98), 
         (130, 172, 179), (115, 209, 128), (204, 79, 135), (136, 126, 185),  (209, 213, 45), (44, 52, 10)]
  
    #plot each boxes
    for box in boxes:
        #add score in label if score=True
        if score:
            label = labels[int(box[-1])+1] + " " + str(round(100 * float(box[-2]),1)) + "%"
        else:
            label = labels[int(box[-1])+1]
        #filter every box under conf threshold if conf threshold setted
        if conf:
            if box[-2] > conf:
                color = colors[int(box[-1])]
                box_label(image, box, label, color)
        else:
            color = colors[int(box[-1])]
            box_label(image, box, label, color)

    #show image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    try:
        import google.colab
        IN_COLAB = True
    except:
        IN_COLAB = False

    if IN_COLAB:
        cv2_imshow(image) #if used in Colab
    else :
        # cv2.imshow(image) #if used in Python
        cv2.imwrite("v3_50.jpg", image)
    

In [11]:
plot_bboxes(image, results[0].boxes.data, conf=0.3)